In [1]:
from pyDO3SE.Defaults import default_processes

In [7]:

from pyDO3SE.Model_State.Model_State import Model_State_Shape
from pyDO3SE.External_State.external_state_loader import load_external_state
from proflow.ProcessRunnerCls import ProcessRunner
from pyDO3SE.Config.config_loader import config_loader
from pyDO3SE.Defaults.state_init_processes import state_init_processes
from pyDO3SE.Defaults.es_init_processes import external_state_init_processes
from pyDO3SE.Defaults.default_processes import full_model_processes

from pyDO3SE.Defaults.config_init_processes import config_init_processes

In [8]:

DEMO_START_DAY = 0
DEMO_END_DAY = 40
HOURS_IN_DAY = 24
DAYS_IN_YEAR = 365
DAYS_TO_RUN = DEMO_END_DAY - DEMO_START_DAY
HOURS_IN_YEAR = HOURS_IN_DAY * DAYS_IN_YEAR
HOURS_TO_RUN = HOURS_IN_DAY * DAYS_TO_RUN
EXT_DATA_COLS = [
    # TODO: This should be based on the config
    'PAR',
    'VPD',
    'Ts_C',
    'u',
    'P',
    'O3',
    'dd',
    'hr',
    'precip',
]

In [9]:
start_day = DEMO_START_DAY
end_day = DEMO_END_DAY
config_location = 'examples/spanish_wheat/spanish_wheat_config.json'
data_location = 'examples/spanish_wheat/spanish_wheat_data.csv'
output_directory = "demo_output"

# == 1. SETUP CONFIG

config = config_loader(config_location, 'json')
process_runner = ProcessRunner(config)

config_amended = process_runner.run_processes(
    config_init_processes(config),
    config)
process_runner.config = config_amended
# == 2. SETUP EXTERNAL STATE
external_state_data = load_external_state(data_location, 'csv', EXT_DATA_COLS)
process_runner.external_state = external_state_data
external_state = process_runner.run_processes(
    external_state_init_processes(start_day, end_day, config),
    external_state_data)

process_runner.external_state = external_state

# TODO: init params

# == SETUP INITIAL STATE
initial_state = Model_State_Shape()
state = process_runner.run_processes(
    state_init_processes(config, start_day, end_day),
    initial_state,
)

# == RUN PROCESSES
processes = full_model_processes(config, start_day, end_day)

In [15]:
p = [p.comment or p.func.__name__ for p in processes]
p

rt',
 'Convert g_sto from umol to mmol, and from H2O to O3',
 'set mean gsto',
 'Scale mean gsto up to bulk gsto',
 'Reset and Calculate the resistance model for O3 over the target canopy',
 'Vd calculation duplicated here just for comparison purposes',
 'Calculate Top Layer Canopy ozone',
 'Calculates Ozone concentration for other layers',
 'Reset and setup Leaf Resistance Model',
 'P-M - Reset Values at beginning of day',
 'Reset previous state',
 'Copy State',
 '<lambda>',
 '<lambda>',
 'set_value',
 'get_row',
 'Calculate the effective temperature',
 'Use height of primary land cover if height method is constant',
 'Calculate height of top of each canopy as a fraction of total canopy height',
 "Use primary land cover's estimate of total LAI and spread over layers and LCs",
 'get LAI_total as sum of all layers and components',
 'Calculate the distribution of LAI between land covers',
 'Calc_SAI_Estimate_total - wheat',
 'Spread single SAI value to layers and LCs',
 'get layer SAI_to